In [1]:
import requests

def get_script(url):
    api_endpoint = "http://127.0.0.1:8010/extract_info"
    response = requests.get(api_endpoint, params={"url": url})
    return response.json()

In [2]:
import time
start_time = time.time()
result = get_script("https://youtu.be/eMlx5fFNoYc?si=V2bsd5GtXo1FhT68")
print(f"실행시간: {time.time() - start_time} seconds")

KeyboardInterrupt: 

In [45]:
result.get("script")[:2]

[{'text': '예예 부장님 예예', 'start': 0.0, 'end': 2.0},
 {'text': '예예 알겠습니다 예예', 'start': 2.0, 'end': 5.0}]

In [46]:
import json
import os
if os.path.exists('script.json'):
    os.remove('script.json')

with open('script.json', 'w', encoding='utf-8') as json_file:
    json.dump(result['script'], json_file, ensure_ascii=False, indent=4)

In [47]:
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings(action='ignore')

load_dotenv()  # .env 파일에서 환경 변수를 로드합니다
api_key = os.getenv("OPENAI_API_KEY")

In [48]:
import json
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_teddynote.callbacks import StreamingCallback
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document

summary_prompt = hub.pull("teddynote/summary-stuff-documents-korean")
summary_prompt.template = 'Please summarize the sentence according to the following REQUEST.\nREQUEST:\n1. Summarize the main points in bullet points in KOREAN.\n2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.\n3. Use various emojis to make the summary more interesting.\n4. Translate the summary into KOREAN if it is written in ENGLISH.\n5. DO NOT translate any technical terms.\n6. DO NOT include any unnecessary information.\n7. Make summary reflect the tone of the original text.\n\nCONTEXT:\n{context}\n\nSUMMARY:"\n'
summary_prompt.pretty_print()


llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    streaming=True,
    temperature=0.7,
    callbacks=[StreamingCallback()],
)

# Step 1: 파일 로드 및 데이터 변환
def load_script(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        script_data = json.load(file)
    return script_data

Please summarize the sentence according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points in KOREAN.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. Translate the summary into KOREAN if it is written in ENGLISH.
5. DO NOT translate any technical terms.
6. DO NOT include any unnecessary information.
7. Make summary reflect the tone of the original text.

CONTEXT:
{context}

SUMMARY:"



In [49]:
from langchain_community.document_loaders import JSONLoader

# JSONLoader로 임시 파일 로드
# jq_schema = '.[] | {content: .text, start_time: .start, end_time: .end}'
jq_schema = ".[].text"
docs = JSONLoader(file_path="script.json", jq_schema=jq_schema,text_content=False).load()

# # 각 Document의 page_content를 디코딩하여 한글로 변환
# for doc in docs:
#     # page_content를 JSON 객체로 변환 후 문자열 디코딩
#     decoded_content = json.loads(doc.page_content) 
#     doc.page_content = decoded_content # JSON 문자열 -> Python dict 변환

In [50]:
docs[0]

Document(metadata={'source': '/Users/jinucho/my_ws/youtube_script_chatbot/script.json', 'seq_num': 1}, page_content='예예 부장님 예예')

In [51]:
stuff_chain = create_stuff_documents_chain(llm, summary_prompt)
answer = stuff_chain.invoke({"context": docs})

- 🍽️ 형부가 밥 먹으러 오라고 재촉함.
- 🤔 처제가 자주 놀러오는 상황.
- 🕒 아침을 늦게 먹어서 배가 고프지 않다고 말함.
- 🍛 카레를 언급하며 몸에 좋은 재료를 넣었다고 설명함.
- 🧪 독극물 여부를 확인하기 위해 은수저 색깔 변화를 언급함.
- 😋 형부가 만든 라면의 맛에 불만을 표함.
- 🍜 라면이 맛없다고 한 이유를 묻는 대화가 이어짐.
- 😂 두 사람의 요리 실력에 대해 농담이 오감.
- 🙄 남은 음식은 송경이가 다 먹어야 한다고 언급함.
- 🔑 비밀번호가 어떻게 바뀌었는지 궁금해함.

In [53]:
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=40)
split_docs = splitter.split_documents(docs)

In [54]:
embeddings = OpenAIEmbeddings()

In [55]:
vec_store = FAISS.from_documents(split_docs, embeddings)

In [56]:
bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k=10

In [57]:
vec_retriever = vec_store.as_retriever( search_kwargs={"k": 10} )

In [58]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[
        bm25_retriever,
        vec_retriever,
    ],
    weights=[0.6, 0.4],
)

In [59]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
chat_prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)


In [60]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
    | chat_prompt
    | llm
    | StrOutputParser()
)

In [65]:
query = "마실것의 재료가 뭔가요?"
ret_docs = ensemble_retriever.invoke(query)
for doc in ret_docs:
    print(doc.page_content)
print("-"*100)
response = chain.invoke(query)

응? 무슨 괜찮네?
어때? 맛은 있지?
이거 냄새 뭐야?
그냥 줘 설거지할게
오 피시방이야?
야 너 얼굴
이거 언니도 하고 먹어
내 국을 피할 줄은 정말 몰랐어
부모님 안부 놓치 마
은희야 이거 마시고 해
뭐하냐?
근데 자기네 음식에 뭐 넣은 거야?
이게 뭐야?
형부 대체 라면은 무슨 짓을 한 거예요?
네가 그렇게 요리를 잘해?
지금 해보자는 거야?
왜 왜 별로야?
----------------------------------------------------------------------------------------------------
모르는 내용입니다.